# 광주 인공지능 사관학교
- - -
- 작성자 : 2반 한지호
- 작성일 : 20.07.15 수
- 3교시 딥러닝 시간에 진행한 '전복 고리 수 추정' 예제 클론 코딩 실습
- 단층 퍼셉트론의 이해를 위해 예제 코드를 강사님과 함께 보며 구조와 그 이유를 파악하였다.
- A, B Line 까지 오늘 진행했고, 다음 수업시간 진도에 맞춰 추가 작성할 예정
- - -
- 수정일 : 20.07.16 목
- C Line ~ D.1 & E.1 까지 진행, 다음 수업시간에 이어 진행할 예정

![flowchart](./flowchart.png)

### 0.0 파이썬 모듈 불러들이기  
- 이 예제는 단층 퍼셉트론의 기본 구조를 파악하기 위해 라이브러리 사용을 최소화한 예제다.

In [1]:
import numpy as np
import csv

np.random.seed(1234)

### 0.1 하이퍼 파라미터 정의  
- 하이퍼 파라미터 : 고정값 (한 번의 실험 사이클 중간에 절대 변경 불가능한 값)  

In [2]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001 # 학습률

### A.1 실험용 메인함수

In [3]:
def abalone_exec(epoch_count=10, mb_szie=10, report=1):
    load_abalone_dataset() # 데이터를 불러들이는 함수
    init_model() # 모델 초기화 함수
    train_and_test(epoch_count, mb_szie, report) # 학습 및 테스트 수행 함수

### B.1 데이터 적재함수 정의

In [4]:
def load_abalone_dataset():
    with open('abalone.csv') as f:
        r = csv.reader(f)
        next(r, None) # next()로 첫 행을 None으로 처리 (원하는 데이터만 출력 = 불필요한 정보를 처리)  
        rows = []
        for row in r:
            rows.append(row)
    
    global data, input_count, output_count # 전역변수 생성
    # 데이터의 입출력 벡터 정보 저장. 이후 크기 지정에 활용. 
    input_count, output_count = 10, 1 #원 핫(3) + 속성(7) = 10, 레이블 = 1
    data = np.zeros(len(rows), input_count+output_count) 
    
    # 원-핫 벡터 처리
    # I = 1,0,0 / M = 0,1,0 / F = 0,0,1
    for n, row in enumerate(rows):
        # data[n, 0~3] = 성별 정보 원 핫 벡터 처리 / 이후 데이터(속성)는 그 뒤에 그대로 복사
        if row[0] == 'I': 
            data[n, 0] = 1 
        if row[0] == 'M': 
            data[n, 1] = 1 
        if row[0] == 'F': 
            data[n, 2] = 1 
        data[n, 3:] = row[1:] 

### B.2 파라미터 초기화 함수 정의

In [5]:
def init_model():
    global weight, bias, input_count, output_count # 전역변수 불러오기 및 생성
    weight = np.random.normal(RND_MEAN, RND_STD, [input_count, output_count]) # 가중치 초기화 : 정규분포를 갖는 난수 생성
    bias = np.zeros([output_count]) # 편향 0으로 초기화

### B.3 학습 및 평가 함수 정의
- 1 epoch = 1 batch (= N mini batch)

In [6]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size) 
    test_x, test_y = get_test_data() 
    
    for epoch in range(epoch_count): # epoch_count 만큼 epoch 반복 수행
        losses, accs = [], [] # 한 차례의 epoch마다 손실과 정확도 저장
        
        for n in range(step_count): # 학습 데이터 크기에 비례하여 (80%) 미니배치 처리된 횟수 만큼 반복 수행
            train_x, train_y = get_train_data(mb_size, n) # 미니배치 마다의 학습 데이터 분할
            loss, acc = run_train(train_x, train_y) # 학습 수행 및 손실과 정확도 산출
            # 미니배치 처리 이후 손실과 정확도를 누적하여 저장 (이후 이 값들을 평균내면 한차례의 'epoch' 처리) 
            losses.append(loss) 
            accs.append(acc) 
            
        if report > 0 and (epoch+1) % report == 0: # 출력 주기 및 테스트 주기 설정
            acc = run_test(test_X, test_y) # 테스트 데이터로 테스트 진행
            print(f'Epoch {epoch+1}: loss={np.mean(losses):5.3f}, accuracy={np.mean(accs):5.3f}/{acc:5.3f}')
            
        final_acc = run_test(test_x, test_y) # 모든 반복이 종료되었을 때, 한 번 더 최종 결과 출력
        print(f'\nFinal Test: final accuracy = {final_acc:5.3f}') 

### C.1~3 학습 및 평가 데이터 획득 함수 정의

In [7]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0]) # 데이터의 순서값을 생성
    np.random.shuffle(shuffle_map) # 데이터를 무작위로 섞어주는 과정 (중복 방지) 
    step_count = int(data.shape[0] * 0.8) // mb_size # 데이터의 80% 기준, 미니배치 사이즈에 의한 1 epoch 당 미니배치 출력 횟수 출력
    test_begin_idx = step_count * mb_size # 학습 데이터와 테스트 데이터의 경계선 인덱스 생성
    return step_count

In [8]:
def get_test_data(): # 테스트 데이터를 일괄 공급하는 함수
    global data, shuffle_map, test_begin_idx, output_count
    test_data = data[shuffle_map[test_begin_idx:]] # 테스트 데이터 생성 
    return test_data[:, :-output_count], test_data[:, -output_count:] # 테스트 데이터의 종속 변수와 독립 변수를 분할하여 반환

In [9]:
def get_train_data(mb_size, nth): # mb_size : 미니배치 크기 / nth : 미니배치 실행 순서
    global data, shuffle_map, test_begin_idx, output_count
    if nth == 0: # 첫 epoch마다 한하여
        np.random.shuffle(shuffle_map[:test_begin_idx]) # 처음부터 경계선까지 인덱스를 섞어준다.
    train_data = data[shuffle_map[mb_size*nth : mb_size*(nth+1)]] # 섞인 인덱스로 ㅣ니배치 크기에 맞게 데이터 분할 및 train_data로 저장
    return train_data[:, :-output_count], test_data[:, -output_count:] # 학습 데이터의 종속 변수와 독립 변수를 분할하여 반환

### C.4 학습 및 평가 데이터 획득 함수 정의
1. 순전파 과정  
  \- 순전파의 뒷 단계 '후처리 과정' : forward_postproc() -> loss
2. 역전파 과정  
  \- 손실에 대한 기울기 : G_loss  
  \- 순전파의 뒷 단계였던 후처리 과정에 대한 역전파 함수 backprop_postproc()가 먼저 호출  
  \- backprop_postproc() -> G_output

In [10]:
def run_train(x, y):
    # 1. 순전파 및 정확도 추출 과정
    output, aux_nn = forward_neuralnet(x) # 신경망 연산 부분 / aux_nn : 입력 벡터 'x' (보조 정보) 
    loss, aux_pp = forward_postproc(output, y) # 신경망 후처리 과정 (손실 함수를 구하는 과정) / aux_pp : 편차 diff
    accuracy = eval_accuracy(output, y) # 정확도를 구하는 과정
    
    # 2. 역전파 과정 : 항상 순전파의 역순으로 수행
    G_loss = 1.0 # 순전파 때 출력이었던 성분의 '손실 함수의 기울기' 
    G_output = backprop_postproc(G_loss, aux_pp) # 손실 함수의 처리 과정인 '평균 제곱 오차'의 역순. 즉, 'MSE의 역전파 처리' 
    # 직접적인 학습이 이뤄지는 부분 (가중치와 편향이 학습률을 활용하여 실제 학습 과정 수행) 
    backprop_neuralnet(G_output, aux_nn) # 입력값에 따른 f(x)에 대한 편미분 과정을 구해주는 내부처리
    
    return loss, accuracy # 손실 값과 정확도를 반환

### C.5 학습 및 평가 데이터 획득 함수 정의

In [11]:
def run_test(x, y): 
    # 순전파 과정 수행, 테스트에서는 역전파를 수행하지 않으므로 보조 정보가 필요하지 않다.
    output, _ = forward_neuralnet(x) # 따라서 두 번째 반환값인 '추가 정보 반환'은 필요 없으므로 '_' 처리
    accuracy = eval_accuracy(output, y) # 최종 정확도 추출
    return accuracy 

### D.1 & E.1 단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의

In [12]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias # 편향이 더해진 입력 벡터와 가중치 벡터에 대한 기본적인 신경망 연산식 / np.matmul() : 두 배열의 행렬곱 연산
    return output, x # 여기서 x는 역전파에 필요한 보조 정보(aux_nn, aux_pp)로 활용

In [13]:
def backprop_neuralnet(G_output, x): # 입력 값에 따른 f(x)에 대한 편미분 과정에서 각각 가중치(G_w)와 편향(G_b)의 손실 기울기 연산
    global weight, bias 
     
    # 가중치에 댈한 손실 기울기 계산
    g_output_w = x.transpose() # 가중치의 손실 기울기를 구하기 위해 필요한 값에 대한 사전 작업 / .transpose() : 해당 편수에 대한 전치 행렬 수행
    G_w = np.matmul(g_output_w, G_output) 
    
    # 편향에 대한 손실 기울기 계산
    G_b = np.sum(G_output, axis=0) 
    
    # 가중치와 편향 갱신
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b